### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import datetime

### Importing data

In [28]:
ais=pd.read_csv(r'vessel-movements-report.csv')

In [29]:
unique_mmsi=ais['MMSI'].unique()

In [30]:
def preprocess(ais):
    '''
    Preprocessing date time column from AIS report
    '''
    ais['DATE TIME (UTC)']=pd.to_datetime(ais['DATE TIME (UTC)'])
    x=ais['DATE TIME (UTC)']-ais['DATE TIME (UTC)'].min()
    y=[]
    for i in range(len(x)):
        y.append(x[i].seconds)
    y=pd.DataFrame(y,columns=['seconds_diff'])

    ais_new=pd.concat([y,ais],axis=1)
    return ais_new

In [31]:
def speed_correction(ais_data, unique_mmsi):  
    '''
    Unreasonable stop during forward movement of vessels
    I.e., position of vessel remains constant even after having positive speed values. 
    If found, first data point out of two will be eliminated from data. Index of such data point will be provided as output.  
    '''
    index = []
    for i in unique_mmsi:
        l=ais_data.loc[ais_data.MMSI==i]
        l=l.reset_index()
        for j in range(len(l)-1):
            if ((l['LATITUDE'][j]==l['LATITUDE'][j+1])&(l['LONGITUDE'][j]==l['LONGITUDE'][j+1])&(l['SPEED']!=0)).all():
                index.append(l.iloc[j,0])
        
        return index

In [23]:
# Preprocessing and AIS data correction step 1

ais_new = preprocess(ais)
ais_new=ais_new.drop(speed_correction(ais_new, unique_mmsi),axis=0)

In [25]:

def distance_jump_correction(ais_data, unique_mmsi):
    '''
    Identifying unreasonable distance jump of any vessel from AIS data.
    To calculate distance between 2 data points, haversine formula is used. 
    Threshold of 15 meters/sec or 29.16 knots has been set for identifying anomalies. 
    Generally, most types of vessels considered in data travel at lower speed than 30 knots. This threshold also depends on geography.
    '''
    r=6378100
    m=[]
    for i in u:
        l=ais_new.loc[ais.MMSI==i]
        l=l.reset_index()
        if len(l)>1:
            for j in range(1,len(l)):
                a=np.radians(l['LATITUDE'][j])
                b=np.radians(l['LATITUDE'][j-1])
                c=np.radians(l['LONGITUDE'][j])
                d=np.radians(l['LONGITUDE'][j-1])
                dis=2*r*np.arcsin(np.sqrt((np.sin((a-b)/2))**2+np.cos(a)*np.cos(b)*(np.sin((c-d)/2))**2))
                sec=l['seconds_diff'][j]-l['seconds_diff'][j-1]
                if (dis/sec>15):
                    m.append(l['index'][j])
    
    return m
            
        

In [26]:
# AIS data correction step 2

ais_new=ais_new.drop(distance_jump_correction(ais_new, unique_mmsi),axis=0)

In [33]:
ais_new

,seconds_diff,DATE TIME (UTC),MMSI,LATITUDE,LONGITUDE,COURSE,SPEED,HEADING,IMO,NAME,CALLSIGN,AISTYPE,A,B,C,D,DRAUGHT,DESTINATION,ETA
0,0,2018-10-23 00:58:01,477166400,18.90669,72.84819,309.5,0.1,215,9827944,HAI SHENG,VRRP9,80,146,31,19,8,5.4,FUJAIRAH,10-26 04:00
1,0,2018-10-23 00:58:01,419000392,18.93804,72.85991,289.5,0.0,511,0,NaN,NaN,0,0,0,0,0,0.0,NaN,00-00 24:60
2,0,2018-10-23 00:58:01,419001175,18.91602,72.91138,218.7,0.2,211,0,MODAK,AWRH,0,0,0,0,0,2.3,MUMBAI,01-01 00:00
3,1,2018-10-23 00:58:02,419000346,18.93963,72.84346,219.3,0.1,511,8216552,MV SUMAYLA ONE,AVNG,90,11,49,3,9,3.2,MUMBAI PLT STN,10-22 06:00
4,2,2018-10-23 00:58:03,636014469,18.94097,72.89935,203.0,0.0,56,9440382,MIKELA P.,A8UB7,89,231,43,32,16,10.3,MUMBAI,10-22 03:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,354,2018-10-23 01:03:55,419001353,18.93939,72.91327,281.8,5.1,277,0,NaN,NaN,0,0,0,0,0,0.0,NaN,00-00 24:60
520,356,2018-10-23 01:03:57,563609000,18.90476,72.87631,304.6,0.0,233,9740249,LANPAN 33,9V2756,32,18,36,6,8,5.1,MUMBAI INDIA,10-21 10:00
521,356,2018-10-23 01:03:57,419000294,18.92682,72.86435,286.9,0.0,204,9547233,SCI PAWAN,AVLI,90,20,46,5,11,5.0,MUMBAI,10-19 18:00
522,357,2018-10-23 01:03:58,419047300,18.95655,72.84838,0.0,0.0,511,8873439,MV RAAVI,AUEJ,90,0,34,0,7,1.5,MUMBAI,05-27 10:00


### AIS data interpolation

In [36]:
# SAR image acquisition time is 25 seconds. 
# The SAR image considered for integration is acquired on 23rd October 2018 during 01:02:56 to 01:03:21 (period of 25 seconds)
# Midpoint of acquisition time is 01:03:08. 
# Difference between AIS data minimum date time and midpoint of SAR acquisition time is 307 seconds.
# midpoint of SAR acquisition time is also called as target interpolation time

sar_date_time = 307

In [48]:

def dead_reckoning_classification(ais_new, sar_date_time):
    '''
    AIS signal of each vessel is categorized into one of the two cases (according to its
    first and last signals present in AIS data) with respect to the target interpolation time (tSAR)
    
    1. AIS signal detected before and after target interpolation time
    2. AIS signal detected only before or only after target interpolation time
    '''
    
    type1 = pd.merge(ais_new.loc[ais_new.seconds_diff>sar_date_time]['MMSI'], ais_new.loc[ais_new.seconds_diff<sar_date_time]['MMSI'], how='inner', on=['MMSI'])['MMSI'].unique()
    type2 = np.array(list(set(ais_new.MMSI).difference(type1)))
    
    return type1, type2

In [49]:
# Linear Interpolation

def calculate_coordinates(lat1, lon1, bearing, speed, seconds, sar_date_time):
    '''
    Linear interpolation is used for type 2 ships. 
    '''
    R = 6371000
    d = speed*0.514444*(sar_date_time-seconds)
    

    lat1 = np.radians(lat1) #Current lat point converted to radians
    lon1 = np.radians(lon1) #Current long point converted to radians

    bearing = np.radians(bearing)

    lat2 = np.arcsin(np.sin(lat1)*np.cos(d/R) + np.cos(lat1)*np.sin(d/R)*np.cos(bearing))

    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(d/R)*np.cos(lat1), np.cos(d/R)-np.sin(lat1)*np.sin(lat2))

    lat2 = np.degrees(lat2)
    lon2 = np.degrees(lon2)

    return (round(lat2, 4), round(lon2, 4))


In [52]:
# Piecewise Hermite

def piecewise_hermite_lat(cor1,cor2,vel1,vel2,t1,t2,theta1,theta2, sar_date_time):
    '''
    Piecewise Hermite interpolation is used for type 1 ships. 
    '''
    a=np.array([[t1**3,t1**2,t1,1], [t2**3,t2**2,t2,1], [3*(t1**2),2*t1,1,0],[3*(t2**2),2*t1,1,0]])
    theta1=np.radians(theta1)
    theta2=np.radians(theta2)
    b=np.array([cor1,cor2,vel1*0.5014*np.sin(theta1)/111000,vel2*0.5014*np.sin(theta2)/111000])
    k=np.linalg.solve(a, b)
    lat=np.dot(k,[sar_date_time**3,sar_date_time**2,314,1])
    return(lat)
    
    
def piecewise_hermite_lon(cor1,cor2,vel1,vel2,t1,t2,theta1,theta2, sar_date_time):
    '''
    Piecewise Hermite interpolation is used for type 1 ships. 
    '''
    a=np.array([[t1**3,t1**2,t1,1],[t2**3,t2**2,t2,1],[3*(t1**2),2*t1,1,0],[3*(t2**2),2*t1,1,0]])
    theta1=np.radians(theta1)
    theta2=np.radians(theta2)
    b=np.array([cor1,cor2,vel1*0.5014*np.cos(theta1)/111000,vel2*0.5014*np.cos(theta2)/111000])
    k=np.linalg.solve(a, b)
    lon=np.dot(k,[sar_date_time**3,sar_date_time**2,sar_date_time,1])
    return(lon)
    

In [54]:
new_data=pd.DataFrame()
type1, type2 = dead_reckoning_classification(ais_new, sar_date_time)

### Interpolating data for type 1 ships using piecewise hermite interpolation

In [56]:
for i in type1:
    
    d=ais_new[(ais_new['MMSI']==i)].reset_index(drop=True)
    index=(abs(d['seconds_diff']-sar_date_time)).idxmin()
    
    if d['seconds_diff'][index]>sar_date_time:
        i2=index
        i1=index-1
    else:
        i2=index+1
        i1=index
    lat1=piecewise_hermite_lat(d['LATITUDE'][i1],d['LATITUDE'][i2],d['SPEED'][i1],d['SPEED'][i2],d['seconds_diff'][i1],
                            d['seconds_diff'][i2],d['COURSE'][i1],d['COURSE'][i2], sar_date_time)
    lon1=piecewise_hermite_lon(d['LONGITUDE'][i1],d['LONGITUDE'][i2],d['SPEED'][i1],d['SPEED'][i2],d['seconds_diff'][i1],
                           d['seconds_diff'][i2],d['COURSE'][i1],d['COURSE'][i2], sar_date_time)
    le=d['A'][0]+d['B'][0]
    wi=d['C'][0]+d['D'][0]

    a=pd.DataFrame([[d['MMSI'][0],lat1,lon1,(d['seconds_diff'][i1]-314),d['SPEED'][i1],le,wi,d['NAME'][0],d['AISTYPE'][0]]],
                   columns=['MMSI','LATITUDE','LONGITUDE','Diff','SPEED_BEFORE','LENGTH','BREADTH','NAME','AISTYPE'])
                   
    new_data=pd.concat([new_data,a],axis=0,ignore_index=True)
    
    
              
              
    
    

### Interpolating data of type 2 ships using linear interpolation

In [57]:

for i in type2:
    l=ais_new[(ais_new['MMSI']==i)].reset_index(drop=True)
    min_index=(abs(l['seconds_diff']-sar_date_time)).idxmin()
    a=calculate_coordinates(l.loc[min_index]['LATITUDE'],l.loc[min_index]['LONGITUDE'],l.loc[min_index]['COURSE'],l.loc[min_index]['SPEED'], l.loc[min_index]['seconds_diff'], sar_date_time)
    b=pd.DataFrame([[l['MMSI'][0],a[0],a[1],(l['seconds_diff'][min_index]-sar_date_time),l['SPEED'][min_index],(l['A'][0]+l['B'][0]),(l['C'][0]+l['D'][0]),l['NAME'][0],l['AISTYPE'][0]]],columns=['MMSI','LATITUDE','LONGITUDE','Diff','SPEED_BEFORE','LENGTH','BREADTH','NAME','AISTYPE'])
    new_data=pd.concat([new_data,b],axis=0,ignore_index=True)

In [58]:
new_data

,MMSI,LATITUDE,LONGITUDE,Diff,SPEED_BEFORE,LENGTH,BREADTH,NAME,AISTYPE
0,419900778,18.889549,72.851226,-37,6.8,0,0,NaN,0
1,565653000,18.944479,72.907363,-35,9.9,268,32,WAN HAI 509,70
2,419001175,18.916093,72.911380,-36,0.0,0,0,MODAK,0
3,419901305,18.953247,72.865884,-33,2.8,70,13,M.V. SHARAVATI,70
4,419000346,18.939646,72.843392,-43,0.0,60,12,MV SUMAYLA ONE,90
...,...,...,...,...,...,...,...,...,...
122,419001071,18.941900,72.082900,-303,1.7,0,0,NaN,0
123,305078000,18.938700,72.841300,-109,0.0,138,21,BBC CAROLINA,70
124,419098100,18.938300,72.858000,-3,0.0,64,18,TAG 7,90
125,419901432,18.887400,72.847200,-39,7.1,0,0,NaN,0


In [59]:
new_data.to_csv('interpolation_of_ships_trial_3.csv',index=False)